# Quality Control for the Caudate Nucleus

In [1]:
suppressPackageStartupMessages({
    library(repr)
    library(here)
    library(dplyr)
    library(ggpubr)
})

## Functions

In [2]:
save_img <- function(image, fn, w, h){
    for(ext in c(".svg", ".pdf", ".png")){
        ggsave(file=paste0(fn, ext), plot=image, width=w, height=h)
    }
}

get_pheno <- function(){
    phenotypes <- data.table::fread(here("input/phenotypes/_m/phenotypes.csv")) |> 
        mutate_if(is.character, as.factor)
    levels(phenotypes$Sex) <- c("Female", "Male")
    return(phenotypes)
}

memPHENO <- memoise::memoise(get_pheno)

## Covariates exmination

### Examine if there are significant differences between diagnosis in continous covariates

In [ ]:
covarsCont = memPHENO() |>
    select(-c('RNum', "Race", "Sex", "Dx", 'Region', 'BrNum', "antipsychotics", 
              "lifetime_antipsych", "Protocol")) |> colnames
options(repr.plot.width=12, repr.plot.height=6)
dir.create("covariate_plots")
for(covar in covarsCont){
    set.seed(20211020)
    flush.console()
    bxp = memPHENO() |> 
        filter(Age > 17, Race %in% c("AA", "EA"), Dx %in% c("CTL", "SZ")) |>
        ggboxplot(x='Sex', y=covar, fill='Sex', add='jitter', palette="npg", 
                  facet.by="Region", legend='bottom', panel.labs.font=list(face='bold'),
                  add.params=list(alpha=0.75), ggtheme=theme_pubr(base_size=20)) + 
        stat_compare_means(comparisons=list(c("Female", "Male")), 
                           aes(label=..p.signif..), method="wilcox.test") +
        font("title", color="black", face="bold") + 
        font("xy.title", face="bold") + #font("xy.text", size=14) + 
        font("legend.title", size=16, face="bold") #font("legend.text", size=14)
    save_img(bxp, paste0("covariate_plots/boxplot_sex_filtered_",covar), w=12, h=6)
    print(bxp)
    bxp = memPHENO() |> 
        filter(Age > 17, Race %in% c("AA", "EA"), Dx %in% c("CTL", "SZ")) |>
        ggboxplot(x='antipsychotics', y=covar, color='Sex', add='jitter', 
                  palette="npg", facet.by="Region", legend='bottom', 
                  ggtheme=theme_pubr(base_size=20)) + 
        stat_compare_means(comparisons=list(c("TRUE", "FALSE")), 
                           aes(label=..p.signif..), method="wilcox.test") +
        font("title", color="black", face="bold") + 
        font("xy.title", face="bold") + 
        font("legend.title", size=16, face="bold")
    save_img(bxp, paste0("covariate_plots/boxplot_sexBYantipsychotics_filtered_",covar), 
             w=12, h=6)
    print(bxp)
}

## Reproducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()